In [1]:
import subprocess
from pydub import AudioSegment

def extract_audio_from_video(video_path, autio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", autio_path]

    subprocess.run(command)

extract_audio_from_video("./files/podcast.mp4", "./files/podcast.mp3",)

track = AudioSegment.from_mp3("./files/podcast.mp3")

track.duration_seconds




4422.426122448979

In [10]:
five_minutes = 5 * 60 * 1000
ten_minutes = 10 * 60 * 1000

import math

chunks = math.ceil(len(track) / ten_minutes)

for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes
    
    chunk = track[start_time:end_time]

    chunk.export(f"./files/chunks/chunk_{i}.mp3", format="mp3")

# first_five = track[:five_minutes]

# first_five.export("./files/first_five.mp3", format="mp3")